## Positive / Negative / Neutral

In [104]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Praise for getting focused over weekend on learning and building side projects"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) positive 0.8953
2) neutral 0.0975
3) negative 0.0072


## Emotion Detection

In [122]:
from transformers import pipeline, TextClassificationPipeline
from pprint import pprint

# classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=None)
sentences = text


model_outputs = classifier(sentences)
model_outputs

[[{'label': 'positive', 'score': 0.8953465819358826},
  {'label': 'neutral', 'score': 0.09747451543807983},
  {'label': 'negative', 'score': 0.007178839761763811}]]

## Praise vs Prayer Request

In [72]:
import sys
sys.path.append('..')
from src.repo.orm import OpenPool, PrayerRequestORM
from src.dto.prayerRequests import PrayerRequest, PrayerRequests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoConfig
from datasets import Dataset
import pandas as pd


# MODEL = "abhi1nandy2/Bible-roberta-base"
MODEL = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)


load_dotenv()
import os
pg_uri = os.environ.get('PRAYERS_PG_DATABASE_URL')
pool = OpenPool(pg_uri)
all_requests = []
with pool() as session:
    prayer_requests = session.query(PrayerRequestORM).all()
    all_requests = [{
        "text": prayer_request.request,
        "label": prayer_request.prayer_type
    } for prayer_request in prayer_requests]
label2id = {
    "Prayer Request": 0,
    "Praise": 1
}
id2label = {v: k for k, v in label2id.items()} 
for request in all_requests:
    request["label"] = label2id[request["label"]]
all_requests = Dataset.from_list(all_requests)

def preprocess_text(request: PrayerRequest):
    return tokenizer(request["text"], truncation=True)
tokenized_requests = all_requests.map(preprocess_text, batched=True)

def stratify_test_train(tokenized_requests, test_size=0.2):
    df_all = pd.DataFrame(tokenized_requests)
    df_train = df_all.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=1-test_size))
    df_test = df_all.drop(df_train.index)
    dataset_train = Dataset.from_pandas(df_train, preserve_index=False)
    dataset_test = Dataset.from_pandas(df_test, preserve_index=False)
    return dataset_train, dataset_test

train_requests, test_requests = stratify_test_train(tokenized_requests)

Map: 100%|██████████| 778/778 [00:00<00:00, 11115.89 examples/s]


In [74]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

accuracy = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer)


model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2, id2label=id2label, label2id=label2id)
training_args = TrainingArguments(
    output_dir="../cache/sentiment_analysis/prayer_type2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
import pickle
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_requests,
    eval_dataset=test_requests,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
with open("../cache/prayers/prayer_or_praise.pkl", 'wb+') as file:
    pickle.dump(model, file)

c:\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
 50%|█████     | 39/78 [05:21<05:28,  8.43s/it]

{'eval_loss': 0.48107343912124634, 'eval_accuracy': 0.7371794871794872, 'eval_runtime': 23.3715, 'eval_samples_per_second': 6.675, 'eval_steps_per_second': 0.428, 'epoch': 1.0}


100%|██████████| 78/78 [10:34<00:00,  6.89s/it]

{'eval_loss': 0.41962629556655884, 'eval_accuracy': 0.8333333333333334, 'eval_runtime': 22.9762, 'eval_samples_per_second': 6.79, 'eval_steps_per_second': 0.435, 'epoch': 2.0}


100%|██████████| 78/78 [10:37<00:00,  8.17s/it]

{'train_runtime': 637.2629, 'train_samples_per_second': 1.952, 'train_steps_per_second': 0.122, 'train_loss': 0.4361897493020082, 'epoch': 2.0}


TrainOutput(global_step=78, training_loss=0.4361897493020082, metrics={'train_runtime': 637.2629, 'train_samples_per_second': 1.952, 'train_steps_per_second': 0.122, 'train_loss': 0.4361897493020082, 'epoch': 2.0})

In [98]:
import tensorflow as tf
import numpy as np
testRequest = Dataset.from_list([{
    "text": "Prayers for sleep, waking up with pain"
}])
testRequest = testRequest.map(preprocess_text, batched=True)
logit = trainer.predict(testRequest).predictions
probabilities = tf.nn.softmax(logit)
# Convert the prediction to use the labels
prediction = []
for i in range(probabilities[0].shape[0]):
    prediction.append({
        "label": id2label[i],
        "score": probabilities[0][i].numpy()
    })
print(prediction)

100%|██████████| 1/1 [00:00<?, ?it/s]

[{'label': 'Prayer Request', 'score': 0.9864389}, {'label': 'Praise', 'score': 0.013561089}]


In [124]:
from transformers import TextClassificationPipeline
import pickle
with open("../cache/prayers/prayer_or_praise.pkl", 'rb') as file:
    prayerModel= pickle.load(file)
classifier = TextClassificationPipeline(model=prayerModel, tokenizer=tokenizer, framework="pt")
classifier("My sleep has been doing so much better!", top_k=None)

[{'label': 'Praise', 'score': 0.5487721562385559},
 {'label': 'Prayer Request', 'score': 0.4512278735637665}]